In [ ]:
import logging
import os
import pandas as pd
from common import indicators
import sys
import numpy as np
import time
from datetime import datetime, timedelta
#from common.get_api_data import *
from common.get_api_data_alpha_vantage import get_data,get_price
from yahoo_fin.stock_info import *



logging.basicConfig(level=logging.INFO)
import common.application as ap
import pytz
#from simulator import place_order as simulator
from common.SlackUtil import sendMessage

app_properties = ap.app_properties
token_mappings = ap.token_mappings
csv_mapping = ap.csv_mapping
keys = ap.keys
api_key = app_properties['api_key']
api_secret = app_properties['api_secret']
token ="INFY"
os.makedirs(token, exist_ok=True)
file_name = "trades.csv"
tz = pytz.timezone('Asia/Kolkata')
logger = logging.getLogger('algo_tester')
# global define
profit = 0.019
stop_loss = 0.0095
historical_data = ""
rsi = 0.00
rsi_slope = 0.00
last_close = 0.00
last_high = 0.00
last_low = 0.00
last_price = get_price(token, 0.00)
current_price = 0.00
holding = ""
order_id = ""
swing = False
flag = ''
interval_int=15
interval=str(interval_int)+"min"

rsi_count=0
sma_triggered=False
extrema_triggered=False
activation=''
def getDateTime():
    datetime_obj_hour_fwd = datetime.now(tz)+timedelta(seconds=75)
    return datetime_obj_hour_fwd


datetime_obj_hour_fwd = getDateTime() + timedelta(hours=1)
from_date = str(datetime_obj_hour_fwd - timedelta(days=14)).split(" ")[0]


In [ ]:
last_price

In [ ]:
import traceback

In [21]:
def reprocess_df(df):
    df = historical_data
    df = indicators.RSI(df,period=14)
    df=indicators.SMA(df, "close", "sma_7", 7)
    df=indicators.SMA(df, "close", "sma_50", 50)
    df['RSI_Trend'] = df['RSI_14'].diff()
    window = 20
    no_of_std = 2
    rolling_mean = df['close'].rolling(window).mean()
    rolling_std = df['close'].rolling(window).std()
    df['ub'] = rolling_mean + (rolling_std * no_of_std)
    df['mb'] = rolling_mean
    df['lb'] = rolling_mean - (rolling_std * no_of_std)
    df['ub_tren']= df['ub'].diff().round(0)
    df['lb_tren']= df['lb'].diff().round(0)
    df['3_ub_point']=df.ub.shift(2).apply(str)+","+    df.ub.shift(1).apply(str)+","+    df.ub.apply(str)
    df['3_lb_point']=df.lb.shift(2).apply(str)+","+    df.lb.shift(1).apply(str)+","+    df.lb.apply(str)
    df['divergence_angle']=df['3_ub_point'].apply(compute_angle)+df['3_lb_point'].apply(compute_angle)
    df['divergence_col']=df.apply(lambda x:  "divergence" if 25 <=x['divergence_angle']<=100 and x['ub_tren']>=0 and x['lb_tren']<=0 and x['ub_tren']!=x['lb_tren'] else '',axis=1)
    df['rsi_indication']=df.RSI_14.apply(lambda x:1 if x>=20 else -2)
    df['rsi_indication_diff']=df.rsi_indication.diff()
    

In [5]:
import numpy as np
def magnitude(v):
    return (v[0]**2+v[1]**2)**(1/2)

def compute_angle(arr):
    arr=arr.split(",")
    p0=(0,float(arr[0]))
    p1=(10,float(arr[1]))
    p2=(20,float(arr[2]))
    v1=[p0[0]-p1[0],p0[1]-p1[1]]
    v2=[p2[0]-p1[0],p2[1]-p1[1]]
    m1=magnitude(v1)
    m2=magnitude(v2)
    return 180-np.arccos(np.dot(v1,v2)/(magnitude(v1)*magnitude(v2)))*180/3.14
    
    

# About to change the direction using RSI

In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
name = "resources/EQUITY_L.csv"
symbols=list(pd.read_csv(name)['SYMBOL'])
trgt_type="resources/res_large_7_sma.csv"
count=0
for symbol in symbols:
    try:
        historical_data = get_data(symbol+".NS" , start_date = '01/01/2019')
        df = historical_data
        df = indicators.RSI(df,period=14)
        df=indicators.SMA(df, "close", "sma_7", 7)
        df=indicators.SMA(df, "close", "sma_50", 50)
        df['RSI_Trend'] = df['RSI_14'].diff()
        window = 20
        no_of_std = 2
        rolling_mean = df['close'].rolling(window).mean()
        rolling_std = df['close'].rolling(window).std()
        df['ub'] = rolling_mean + (rolling_std * no_of_std)
        df['mb'] = rolling_mean
        df['lb'] = rolling_mean - (rolling_std * no_of_std)
        df['ub_tren']= df['ub'].diff().round(0)
        df['lb_tren']= df['lb'].diff().round(0)
        df['3_ub_point']=df.ub.shift(2).apply(str)+","+    df.ub.shift(1).apply(str)+","+    df.ub.apply(str)
        df['3_lb_point']=df.lb.shift(2).apply(str)+","+    df.lb.shift(1).apply(str)+","+    df.lb.apply(str)
        df['divergence_angle']=df['3_ub_point'].apply(compute_angle)+df['3_lb_point'].apply(compute_angle)
        df['divergence_col']=df.apply(lambda x:  "divergence" if 25 <=x['divergence_angle']<=100 and x['ub_tren']>=0 and x['lb_tren']<=0 and x['ub_tren']!=x['lb_tren'] else '',axis=1)
        df['rsi_indication']=df.RSI_14.apply(lambda x:1 if x<=80 else -2)
        df['rsi_indication_diff']=df.rsi_indication.diff()
        count=count+1
        print(count)
        if(-3 in list(df[-7:].rsi_indication_diff)) :
            print("inside",symbol)
            df[-1:].to_csv(trgt_type,mode='a',index=False,header=False)
    except Exception as e:
        print("Error ",e)
        #print(traceback.format_exc())
        print("Not present symbol",symbol)


# Checking Support and ressistnace more than a week

In [33]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
name = "resources/ind_nifty100list.csv"
symbols=list(pd.read_csv(name)['Symbol'])
trgt_type="resources/res_large_7_sma.csv"
count=0
for symbol in symbols:
    try:
        historical_data = get_data(symbol+".NS" , start_date = '01/01/2019')
        df=historical_data
        reprocess_df(df)
        count=count+1
        print(count)
        closed_7thday=float(df[-7:-6].close)
        closed_latest_day=float(df[-1:].close)
        if(1.3>=closed_7thday/closed_latest_day>=0.7 and int(df[-1:].RSI_14)>=70):
            print("inside",symbol)
            df[-1:]['ticker'].to_csv(trgt_type,mode='a',index=False,header=False)
    except Exception as e:
        print("Error ",e)
        #print(traceback.format_exc())
        print("Not present symbol",symbol)

        
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
inside RELIANCE
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
name = "resources/EQUITY_L.csv"
symbols=list(pd.read_csv(name)['SYMBOL'])[0:1]
trgt_type="resources/res_1.csv"
count=0
symbol='CINELINE'
historical_data = get_data(symbol+".NS" , start_date = '01/01/2019')
df = historical_data
df = indicators.RSI(df,period=14)
df=indicators.SMA(df, "close", "sma_7", 7)
df['RSI_Trend'] = df['RSI_14'].diff()
window = 20
no_of_std = 2
rolling_mean = df['close'].rolling(window).mean()
rolling_std = df['close'].rolling(window).std()
df['ub'] = rolling_mean + (rolling_std * no_of_std)
df['mb'] = rolling_mean
df['lb'] = rolling_mean - (rolling_std * no_of_std)
df['ub_tren']= df['ub'].diff().round(0)
df['lb_tren']= df['lb'].diff().round(0)
df['3_ub_point']=df.ub.shift(2).apply(str)+","+    df.ub.shift(1).apply(str)+","+    df.ub.apply(str)
df['3_lb_point']=df.lb.shift(2).apply(str)+","+    df.lb.shift(1).apply(str)+","+    df.lb.apply(str)
df['divergence_angle']=df['3_ub_point'].apply(compute_angle)+df['3_lb_point'].apply(compute_angle)
df['divergence_col']=df.apply(lambda x:  "divergence" if 25 <=x['divergence_angle']<=100 and x['ub_tren']>=0 and x['lb_tren']<=0 and x['ub_tren']!=x['lb_tren'] else '',axis=1)
df['rsi_indication']=df.RSI_14.apply(lambda x:1 if x<=30 else -2)
df['rsi_indication_diff']=df.rsi_indication.diff()
count=count+1
print(count)
if(-3 in list(df[-:].rsi_indication_diff)) 
    print("inside",symbol)
    df[-1:].to_csv(trgt_type,mode='a',index=False,header=False)



In [ ]:
df[-7:]

In [ ]:
get_analysts_info('nflx')

In [ ]:
try:
    raise("dfhjksdfl")
except Exception as e:
    print(str(e))

In [ ]:
global historical_data, rsi, rsi_slope, last_close, swing,last_high,last_low,sma,ub,mb,lb,band_diff_trend,current_price
swing = False
name = "resources/EQUITY_L.csv"
symbols=list(pd.read_csv(name)['SYMBOL'])
trgt_type="resources/res_large_7_sma.csv"
count=0
for symbol in symbols:
    try:
        last_price = get_price(symbol, 0.00)
        if(5<=last_price<=10):
            print(symbol)
    except:
        pass

In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark import StorageLevel
from datetime import timedelta
import datetime
import csv
import os.path
from datetime import datetime as d
import re
import subprocess as sp

back_date = 4
today = datetime.datetime.now().date()
start_date = today - timedelta(days=(today.weekday()-3)%7+7)

In [ ]:
str(start_date)

In [ ]:
data=spark.read.csv(path,spep='\t',header=True)
d_temp=data.withColumn("Date",lit(start_date))
d_temp.write.csv("temp")
rm -r path
mv temp path

In [34]:
import requests

url = "https://quotient.p.rapidapi.com/intradayp"

querystring = {"end":"2020-04-21 12%3A00","adjust":"false","interval":"5","start":"2020-04-21 10%3A00","symbol":"MSFT"}

headers = {
    'x-rapidapi-host': "quotient.p.rapidapi.com",
    'x-rapidapi-key': "977bb6ce7emsh786c927aabb4013p1a4f36jsn05b616c89d51"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}


In [37]:
token='INFY'
key="XAFFZFOLZ8JV3HOO"
r=requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NSE:"+str(token)+"&interval=5min&apikey="+key+"&datatype=csv")
print(r.content)
c=pd.read_csv(io.StringIO(r.content.decode('utf-8')))

b'timestamp,open,high,low,close,volume\r\n2020-05-06 00:58:56,678.6000,678.6000,678.6000,678.6000,0\r\n2020-05-06 00:10:00,670.2500,670.6000,669.0000,669.2000,58139\r\n2020-05-06 00:05:00,670.8000,672.5000,669.5000,670.0000,49552\r\n2020-05-06 00:00:00,669.7200,672.8900,668.9900,670.9700,105101\r\n2020-05-05 23:55:00,669.5500,671.1500,668.2000,669.9500,86773\r\n2020-05-05 23:50:00,670.8000,672.2500,668.0000,669.5000,116019\r\n2020-05-05 23:45:00,678.8500,678.8500,671.0000,671.3000,0\r\n2020-05-05 05:55:00,675.4000,675.5500,673.7000,673.7000,270643\r\n2020-05-05 05:50:00,674.5000,676.1000,674.4000,675.6000,384772\r\n2020-05-05 05:45:00,674.1500,675.0000,672.2000,674.3500,421013\r\n2020-05-05 05:40:00,673.1000,676.0000,672.6000,674.2500,457022\r\n2020-05-05 05:35:00,670.6500,674.7500,670.5000,673.0500,379730\r\n2020-05-05 05:30:00,676.3000,676.3000,670.0000,670.6500,508390\r\n2020-05-05 05:25:00,677.1500,677.2000,675.0500,676.3500,108111\r\n2020-05-05 05:20:00,677.7000,678.0000,676.3000,

In [54]:
c['timestamp']=pd.to_datetime(c.timestamp)+pd.to_timedelta(570, unit='m')
k=c.sort_values("timestamp",ascending=True)

In [62]:
int(str(k[0:1].timestamp).split("\n")[0].split(" ")[-1].split(":")[0])%5==0


False

In [74]:
k.shape[0]+1

101

In [76]:
k['seq']=pd.Series([i for i in range(1,101)][::-1])

In [77]:
range(1,100)

range(1, 100)

In [128]:
token='SBIN'
key="XAFFZFOLZ8JV3HOO"
r=requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NSE:"+str(token)+"&interval=5min&apikey="+key+"&datatype=csv")
print(r.content)
c=pd.read_csv(io.StringIO(r.content.decode('utf-8')))
c['timestamp']=pd.to_datetime(c.timestamp)+pd.to_timedelta(570, unit='m')
k=c.sort_values("timestamp",ascending=True)
if(int(str(k[0:1].timestamp).split("\n")[0].split(" ")[-1].split(":")[1])%10==0):
    k=k[1:]
k['open_s']=k.open.shift(-1)
k['close_s']=k.close.shift(-1)
k['high_s']=k.high.shift(-1)
k['low_s']=k.low.shift(-1)
k['volume_s']=k.volume.shift(-1)
k['seq']=pd.Series([i for i in range(1,k.shape[0]+1)][::-1])

b'timestamp,open,high,low,close,volume\r\n2020-05-06 03:30:04,170.6000,170.6000,170.6000,170.6000,0\r\n2020-05-06 00:10:00,168.5000,169.2000,168.1000,169.1000,1071512\r\n2020-05-06 00:05:00,167.9500,168.5500,167.6500,168.5000,895469\r\n2020-05-06 00:00:00,167.7100,168.2000,167.6000,167.8600,1121977\r\n2020-05-05 23:55:00,167.9500,168.1000,167.2500,167.6500,1111874\r\n2020-05-05 23:50:00,167.7500,168.1500,166.6000,167.9000,2236711\r\n2020-05-05 23:45:00,171.0000,171.2500,167.1500,167.7500,0\r\n2020-05-05 05:55:00,171.0500,171.4500,170.9500,171.4000,1540944\r\n2020-05-05 05:50:00,170.5500,171.0000,170.0000,171.0000,1866499\r\n2020-05-05 05:45:00,170.1000,170.6000,170.0500,170.5000,1684304\r\n2020-05-05 05:40:00,170.4500,170.6500,169.8500,170.1500,2074531\r\n2020-05-05 05:35:00,170.1000,170.5000,169.9000,170.5000,1336086\r\n2020-05-05 05:30:00,170.0500,170.6000,169.7500,170.2000,1718308\r\n2020-05-05 05:25:00,169.5000,170.6500,169.0000,170.1500,3372737\r\n2020-05-05 05:20:00,171.4000,171.

In [123]:
k['seq']=pd.Series([i for i in range(1,101)][::-1])

In [124]:
k=k[k.apply(lambda x:x['seq']%2==1,axis=1)]

In [125]:
k

,timestamp,open,high,low,close,volume,open_s,close_s,high_s,low_s,volume_s,seq
97,2020-05-04 14:10:00,178.55,178.90,178.20,178.80,518145,178.70,178.70,178.90,178.50,317783.0,3
95,2020-05-04 14:20:00,178.70,179.25,178.60,179.20,389498,179.20,178.65,179.25,178.60,419416.0,5
93,2020-05-04 14:30:00,178.66,178.91,178.51,178.61,427941,178.60,178.30,178.60,178.20,557651.0,7
91,2020-05-04 14:40:00,178.35,178.45,178.05,178.35,626558,178.35,178.25,178.35,178.00,742660.0,9
89,2020-05-04 14:50:00,178.15,178.55,178.00,178.50,810809,178.50,178.50,178.85,178.40,629477.0,11
87,2020-05-04 15:00:00,178.60,178.80,178.50,178.55,632247,178.55,178.70,178.75,178.50,711237.0,13
85,2020-05-04 15:10:00,178.75,178.90,178.50,178.80,1127991,178.75,179.00,179.10,178.70,1170699.0,15
83,2020-05-04 15:20:00,179.00,179.20,178.75,179.20,1031650,179.20,179.25,179.25,179.05,733834.0,17
81,2020-05-05 09:15:00,180.50,181.20,180.00,180.90,0,180.90,180.55,181.20,180.55,759307.0,19
79,2020-05-05 09:25:00,180.60,180.60,179.55,179.65,1091753,179.61,179.76,180.26,179.56,772627.0,21


In [129]:
k=k[k.apply(lambda x:x['seq']%2==1,axis=1)]

In [130]:
k

,timestamp,open,high,low,close,volume,open_s,close_s,high_s,low_s,volume_s,seq
98,2020-05-04 14:05:00,178.70,178.80,178.40,178.50,785171,178.55,178.80,178.90,178.20,518145.0,1
96,2020-05-04 14:15:00,178.70,178.90,178.50,178.70,317783,178.70,179.20,179.25,178.60,389498.0,3
94,2020-05-04 14:25:00,179.20,179.25,178.60,178.65,419416,178.66,178.61,178.91,178.51,427941.0,5
92,2020-05-04 14:35:00,178.60,178.60,178.20,178.30,557651,178.35,178.35,178.45,178.05,626558.0,7
90,2020-05-04 14:45:00,178.35,178.35,178.00,178.25,742660,178.15,178.50,178.55,178.00,810809.0,9
88,2020-05-04 14:55:00,178.50,178.85,178.40,178.50,629477,178.60,178.55,178.80,178.50,632247.0,11
86,2020-05-04 15:05:00,178.55,178.75,178.50,178.70,711237,178.75,178.80,178.90,178.50,1127991.0,13
84,2020-05-04 15:15:00,178.75,179.10,178.70,179.00,1170699,179.00,179.20,179.20,178.75,1031650.0,15
82,2020-05-04 15:25:00,179.20,179.25,179.05,179.25,733834,180.50,180.90,181.20,180.00,0.0,17
80,2020-05-05 09:20:00,180.90,181.20,180.55,180.55,759307,180.60,179.65,180.60,179.55,1091753.0,19


In [131]:
!pip install alpha_vantage

  Stored in directory: C:\Users\shivam.gupta\AppData\Local\pip\Cache\wheels\3a\fa\05\80b0afcbd2a4de8bd2a0c5cd77084e3d3a9cc12c48be9b7f84
Successfully built alpha-vantage


In [143]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
ts = TimeSeries(key=key,output_format='pandas')
data, meta_data = ts.get_intraday(symbol='NSE:INFY',interval='5min', outputsize='full')
c=data
c['timestamp']=c.index
c['timestamp']=pd.to_datetime(c.timestamp)+pd.to_timedelta(570, unit='m')

In [144]:
data

,1. open,2. high,3. low,4. close,5. volume,timestamp
date,,,,,,
2020-04-14 23:45:00,651.40,652.85,641.60,647.15,336661.0,2020-04-15 09:15:00
2020-04-14 23:50:00,646.95,650.45,646.00,646.15,546439.0,2020-04-15 09:20:00
2020-04-14 23:55:00,646.05,646.05,641.05,641.35,330726.0,2020-04-15 09:25:00
2020-04-15 00:00:00,641.25,643.80,641.05,643.25,308453.0,2020-04-15 09:30:00
2020-04-15 00:05:00,643.10,645.00,642.55,644.90,205725.0,2020-04-15 09:35:00
2020-04-15 00:10:00,644.90,647.35,644.20,644.90,195386.0,2020-04-15 09:40:00
2020-04-15 00:15:00,644.90,648.00,644.85,647.35,205715.0,2020-04-15 09:45:00
2020-04-15 00:20:00,647.40,650.00,646.90,649.60,345509.0,2020-04-15 09:50:00
2020-04-15 00:25:00,649.65,650.70,648.75,650.65,384293.0,2020-04-15 09:55:00
